In [10]:
pip install selenium bs4 beautifulsoup4 pandas webdriver_manager


Note: you may need to restart the kernel to use updated packages.


In [4]:
import time
from datetime import datetime
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Function to get topic from the user
def get_topic_url(topic):
    base_url = "https://economictimes.indiatimes.com/topic/"
    topic_url = base_url + topic.replace(" ", "-") + "/news"
    return topic_url

# Get topic from the user
topic = input("Enter the topic to scrape: ")
url = get_topic_url(topic)

# Set up conditions
max_articles = input("Enter the maximum number of articles to scrape (leave blank for no limit): ")
max_articles = int(max_articles) if max_articles else None

max_date_str = input("Enter the latest date to scrape articles (in format YYYY-MM-DD, leave blank for no limit): ")
max_date = datetime.strptime(max_date_str, "%Y-%m-%d") if max_date_str else None

# Set up Selenium WebDriver
options = Options()
options.headless = True
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get(url)

# Initialize a list to store the scraped data
data = []

try:
    # Scroll and click "Load More" until no more content is loaded
    while True:
        # Parse the HTML content of the webpage
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, "html.parser")

        # Find all article elements
        articles = soup.find_all("div", class_="clr flt topicstry story_list")

        # Extract data from each article element
        for article in articles:
            # Find the title and link
            title_tag = article.find("h2").find("a")
            title = title_tag.get_text(strip=True)
            link = "https://economictimes.indiatimes.com" + title_tag["href"]

            # Find the description
            description_tag = article.find("p", class_="wrapLines l3")
            description = description_tag.get_text(strip=True) if description_tag else "No description"

            # Find the time of publication
            time_tag = article.find("time")
            time_value = time_tag.get_text(strip=True) if time_tag else "No time"
            article_date = datetime.strptime(time_value, "%d %b, %Y, %I:%M %p IST") if time_tag else None

            # Append the extracted data to the list
            data.append({
                "Title": title,
                "Link": link,
                "Description": description,
                "Time": time_value
            })

            # Check the conditions
            if max_articles and len(data) >= max_articles:
                break
            if max_date and article_date and article_date < max_date:
                break

        # Check the conditions after processing the articles
        if max_articles and len(data) >= max_articles:
            break
        if max_date and article_date and article_date < max_date:
            break
        
        # Scroll all the way to the bottom of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # wait for the page to load new content

        # Try to click the "Load More" button using JavaScript
        try:
            load_more_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'jsTopicLoadMore'))
            )
            driver.execute_script("arguments[0].click();", load_more_button)
            time.sleep(3)  # wait for new content to load
        except:
            print("No more 'Load More' button found or it could not be clicked. Stopping the scroll.")
            break  # no more "Load More" button, exit loop
except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Close the driver
    driver.quit()

    # Convert the data into a DataFrame
    df = pd.DataFrame(data)

    # Display the DataFrame
    print(df.head())

    # Construct the filename
    filename = f"{topic.replace(' ', '_')}_articles"
    if max_articles:
        filename += f"_max{max_articles}"
    else:
        filename += f"_max_articles"
    if max_date:
        filename += f"_end{max_date.strftime('%Y%m%d')}"
    else:
        filename += f"_max_date"
    filename += ".csv"

    # Save the DataFrame to a CSV file
    df.to_csv(filename, index=False)

    print(f"Data saved to {filename}")

An error occurred: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=126.0.6478.127)
Stacktrace:
	GetHandleVerifier [0x00D8C1C3+27395]
	(No symbol) [0x00D23DC4]
	(No symbol) [0x00C21B7F]
	(No symbol) [0x00C0C8E8]
	(No symbol) [0x00C0C809]
	(No symbol) [0x00C23DB0]
	(No symbol) [0x00C9C3F6]
	(No symbol) [0x00C83736]
	(No symbol) [0x00C57541]
	(No symbol) [0x00C580BD]
	GetHandleVerifier [0x01043A93+2876371]
	GetHandleVerifier [0x01097F5D+3221661]
	GetHandleVerifier [0x00E0D634+556916]
	GetHandleVerifier [0x00E1474C+585868]
	(No symbol) [0x00D2CE04]
	(No symbol) [0x00D29818]
	(No symbol) [0x00D299B7]
	(No symbol) [0x00D1BF0E]
	BaseThreadInitThunk [0x76FEFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x778A80CE+286]
	RtlGetAppContainerNamedObjectPath [0x778A809E+238]

                                               Title  \
0  NexGen Energia plans to invest Rs 15,000 cr ov...   
1  JSW Ene